In [30]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("stock.db")
df = pd.read_sql_query(
    """
                        SELECT STOCK_CODE,
                               STOCK_YEAR,
                               STOCK_DATA
                          FROM vcsc_financial_ratio
                         WHERE stock_factor = 'EPS (VND)' and stock_year not in ('TTM Q1/2018')
                         GROUP BY stock_code,
                                  stock_year,
                                  stock_data;           
                        """, conn)
df['STOCK_YEAR'] = pd.to_numeric(df['STOCK_YEAR'])
df['STOCK_DATA'] = df['STOCK_DATA'].str.replace('-', '')
df['STOCK_DATA'] = df['STOCK_DATA'].str.replace(',', '')


df['STOCK_DATA'] = pd.to_numeric(df['STOCK_DATA'])


df['D'] = df.groupby("STOCK_YEAR")["STOCK_DATA"].rank("dense", ascending=False)
df['C'] = pd.qcut(df['STOCK_DATA'], 10, labels=list(range(10)))

df["quantiles"] = df.groupby("STOCK_YEAR")["STOCK_DATA"].transform(pd.qcut,10,list(range(10)))
df.loc[df['STOCK_CODE'] == 'AAA']

,STOCK_CODE,STOCK_YEAR,STOCK_DATA,D,C,quantiles
0,AAA,2008,3466.0,236.0,7,7.0
1,AAA,2009,5716.0,151.0,8,8.0
2,AAA,2010,9114.0,54.0,9,9.0
3,AAA,2011,6614.0,89.0,9,9.0
4,AAA,2012,2785.0,331.0,6,7.0
5,AAA,2013,2775.0,324.0,6,7.0
6,AAA,2014,1186.0,707.0,3,4.0
7,AAA,2015,740.0,851.0,2,3.0
8,AAA,2016,2293.0,418.0,6,7.0
9,AAA,2017,2672.0,403.0,6,7.0


In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("stock.db")
df = pd.read_sql_query(
    """
                        SELECT STOCK_CODE,
                               STOCK_YEAR,
                               STOCK_DATA
                          FROM vcsc_financial_ratio
                         WHERE stock_factor = 'EPS (VND)' and stock_year not in ('TTM Q1/2018')
                         GROUP BY stock_code,
                                  stock_year,
                                  stock_data;           
                        """, conn)
df['STOCK_YEAR'] = pd.to_numeric(df['STOCK_YEAR'])

df2 = pd.read_sql_query(
    """
                        select STOCK_CODE, STOCK_DATE, STOCK_CLOSE from vcsc_stock_price;
                        """, conn)

df2['STOCK_DATE'] = pd.to_datetime(df2['STOCK_DATE'], format='%m/%d/%Y')

df2['STOCK_YEAR'] = df2['STOCK_DATE'].dt.year
df2['STOCK_MONTH'] = df2['STOCK_DATE'].dt.month
df2['STOCK_DAY'] = df2['STOCK_DATE'].dt.day

df2['QUARTER'] = df2['STOCK_DATE'].dt.to_period("Q")

gb = df2.groupby([df2['STOCK_CODE'], df2['STOCK_YEAR'], df2['STOCK_MONTH']]).agg({
    'STOCK_CLOSE':
    'first'
})

df3 = df.groupby([df['STOCK_CODE'], df['STOCK_YEAR']]).agg({'STOCK_DATA': 'first'})




result = pd.merge(df2, df, on=['STOCK_CODE', 'STOCK_YEAR'], how='left')

result2 = result.sort_values(['STOCK_DATE'], ascending=[False]).groupby(['STOCK_CODE', 'STOCK_YEAR', 'STOCK_MONTH']).first()
result2

dfx = df.copy()
dfx.index += 1 
dfx

dfy = df.copy()
dfy.index += 2 
dfy


#results = pd.merge(dfy, dfx, df, left_index=True, right_index=True)
#results

result3 = pd.concat([df, dfx, dfy], axis=1, sort=False)

result

dfg = df2.sort_values(['STOCK_DATE'], ascending=[False]).groupby(['STOCK_CODE', 'QUARTER']).first()

dfg




#df.pivot(index='stock_code', columns='stock_year', values='stock_data')
#conn.close()

,STOCK_CODE,STOCK_YEAR,STOCK_DATA
1,AAA,2008,"3,466"
2,AAA,2009,"5,716"
3,AAA,2010,"9,114"
4,AAA,2011,"6,614"
5,AAA,2012,"2,785"
6,AAA,2013,"2,775"
7,AAA,2014,"1,186"
8,AAA,2015,740
9,AAA,2016,"2,293"
10,AAA,2017,"2,672"


In [ ]:

ds_price['INDEX'] = range(1, len(ds_price) + 1)
ds_price

ds_price1 = ds_price.copy()
ds_price1['INDEX'] += 1 

ds_price2 = ds_price1.copy()
ds_price2['INDEX'] += 1 

#ds_pricef = pd.concat([ds_price, ds_price1, ds_price2], axis=1, keys='INDEX')
#ds_pricef = pd.merge(ds_price, ds_price1, on=['INDEX'], right_index=True)
ds_pricef = ds_price.merge(ds_price1, on='INDEX', right_index=True).merge(ds_price2, on='INDEX', right_index=True)

ds_pricef

In [179]:
import sqlite3
import numpy as np
import pandas as pd
import itertools
import time

st = time.time()

conn = sqlite3.connect("stock.db")
ds_ratio = pd.read_sql_query('select STOCK_CODE, STOCK_FACTOR, STOCK_YEAR, STOCK_DATA from VCSC_FINANCIAL_RATIO', conn)
ds_price = pd.read_sql_query('select STOCK_CODE, STOCK_DATE, STOCK_CLOSE from VCSC_STOCK_PRICE', conn)
conn.close()

print("1--- %s seconds ---" % (time.time() - st))
st = time.time()


ds_weight = np.arange(0.0, 1.01, 0.01)
#ds_weight = np.around(numbers, decimals=2)
ds_weight = [x for x in itertools.permutations(ds_weight, 2) if sum(x) == 1]
ds_weight = pd.DataFrame(ds_weight,columns=('W1','W2'))

ds_ratio = ds_ratio.loc[ds_ratio['STOCK_FACTOR'].isin(['EPS (VND)', 'BVS'])]
ds_ratio = ds_ratio.loc[ds_ratio['STOCK_YEAR'].str.contains('TTM')==False]
ds_ratio['STOCK_DATA'] = ds_ratio['STOCK_DATA'].str.replace('-', '')
ds_ratio['STOCK_DATA'] = ds_ratio['STOCK_DATA'].str.replace(',', '')
ds_ratio['STOCK_DATA'] = pd.to_numeric(ds_ratio['STOCK_DATA'])
ds_ratio['STOCK_YEAR'] = pd.to_numeric(ds_ratio['STOCK_YEAR'])

ds_price['STOCK_DATE'] = pd.to_datetime(ds_price['STOCK_DATE'], format='%m/%d/%Y')
ds_price['STOCK_YEAR'] = ds_price['STOCK_DATE'].dt.year
ds_price['STOCK_QUARTER'] = ds_price['STOCK_YEAR'].astype('str') + ds_price['STOCK_DATE'].dt.quarter.astype('str')
ds_price['STOCK_CLOSE'] = ds_price['STOCK_CLOSE'].str.replace(',', '').astype(float)

ds_price = ds_price.sort_values(['STOCK_DATE'], ascending=[False]).groupby(['STOCK_CODE', 'STOCK_QUARTER']).nth(0)
ds_price['NEXT-PRICE-Q'] = ds_price.groupby(level=0)['STOCK_CLOSE'].shift(-1)
ds_price = ds_price.sort_values(['STOCK_DATE'], ascending=[False]).groupby(['STOCK_CODE', 'STOCK_YEAR']).nth(0)
ds_price['NEXT-PRICE-Y'] = ds_price.groupby(level=0)['STOCK_CLOSE'].shift(-1)

ds_ratio = ds_ratio.merge(ds_price, left_on=['STOCK_CODE', 'STOCK_YEAR'], right_on=['STOCK_CODE', 'STOCK_YEAR'], how='right')
ds_ratio['XP'] = ds_ratio['STOCK_DATA'].div(ds_ratio['NEXT-PRICE-Q'])
ds_ratio['RETURN'] = ds_ratio['NEXT-PRICE-Y'].div(ds_ratio['NEXT-PRICE-Q'])
ds_ratio['RANK'] = ds_ratio.groupby(['STOCK_FACTOR', 'STOCK_YEAR'])['XP'].rank('dense', ascending=False)

ds_p = ds_ratio.pivot_table(index=['STOCK_YEAR', 'STOCK_CODE'], columns='STOCK_FACTOR', values=['RANK', 'RETURN'])
ds_p.columns = [' '.join(col).strip() for col in ds_p.columns.values]
ds_p = ds_p.reset_index()

ds_p['TMP'] = 1
ds_weight['TMP'] = 1
ds_p = pd.merge(ds_p, ds_weight, on='TMP', how='outer')
ds_p.fillna(0, inplace=True)

ds_p['RANK'] = ds_p['RANK BVS'] * ds_p['W1'] + ds_p['RANK EPS (VND)'] * ds_p['W2']
ds_p['DECILE'] = (ds_p.groupby(['W1', 'W2', 'STOCK_YEAR'])['RANK']
                                .apply(lambda x: pd.qcut(x,10,duplicates='drop',labels=False)))
ds_p = ds_p.loc[ds_p['DECILE'] == 9.0]

ds_p['RETURN'] = ds_p['RETURN BVS']
ds_p = ds_p.drop(columns=['STOCK_CODE', 'RANK BVS', 'RANK EPS (VND)', 'RETURN BVS', 'RETURN EPS (VND)'])
ds_p = ds_p.drop(columns=['TMP', 'RANK', 'DECILE'])

s6 = ds_p.groupby(['W1', 'W2', 'STOCK_YEAR']).agg({'RETURN':'mean'})
s6 = s6.reset_index()

s6.pivot_table(index=(['W1', 'W2']),columns='STOCK_YEAR', values='RETURN').sort_values(by=2017, ascending=False)


1--- 2.6288795471191406 seconds ---


,STOCK_YEAR,2011,2012,2013,2014,2015,2016,2017
W1,W2,,,,,,,
0.15,0.85,0.788023,1.044578,0.961199,0.924771,0.891273,1.100610,0.936920
0.14,0.86,0.786312,1.044578,0.960226,0.925763,0.884312,1.106657,0.936300
0.16,0.84,0.784694,1.031796,0.961882,0.932591,0.891273,1.103082,0.935427
0.17,0.83,0.784694,1.029625,0.977333,0.937905,0.889371,1.070753,0.932912
0.13,0.87,0.786312,1.039214,0.975290,0.917472,0.875864,1.109362,0.929838
0.06,0.94,0.781471,1.078101,1.005463,0.913474,0.872457,1.093943,0.929481
0.07,0.93,0.779689,1.070282,1.005463,0.918312,0.858684,1.091750,0.929481
0.10,0.90,0.777459,1.051323,0.999714,0.916592,0.871719,1.096757,0.928889
0.11,0.89,0.779003,1.051323,0.986880,0.910630,0.865438,1.100185,0.928807


In [25]:
import sqlite3
import numpy as np
import pandas as pd
import itertools
import time

st = time.time()

conn = sqlite3.connect("stock.db")
ds_ratio = pd.read_sql_query('select STOCK_CODE, STOCK_FACTOR, STOCK_YEAR, STOCK_DATA from VCSC_FINANCIAL_RATIO', conn)
ds_price = pd.read_sql_query('select STOCK_CODE, STOCK_DATE, STOCK_CLOSE from VCSC_STOCK_PRICE', conn)
conn.close()

print("1--- %s seconds ---" % (time.time() - st))
st = time.time()


#ds_weight = np.arange(0.0, 1.01, 0.01)
#ds_weight = np.around(numbers, decimals=2)
#ds_weight = [x for x in itertools.permutations(ds_weight, 1) if sum(x) == 1]
#ds_weight = pd.DataFrame(ds_weight,columns=('W1','W2'))

ds_weight = pd.DataFrame(data={'W1': [1.0]})

ds_ratio = ds_ratio.loc[ds_ratio['STOCK_FACTOR'].isin(['BVS'])]
ds_ratio = ds_ratio.loc[ds_ratio['STOCK_YEAR'].str.contains('TTM')==False]
ds_ratio['STOCK_DATA'] = ds_ratio['STOCK_DATA'].str.replace('-', '')
ds_ratio['STOCK_DATA'] = ds_ratio['STOCK_DATA'].str.replace(',', '')
ds_ratio['STOCK_DATA'] = pd.to_numeric(ds_ratio['STOCK_DATA'])
ds_ratio['STOCK_YEAR'] = pd.to_numeric(ds_ratio['STOCK_YEAR'])

ds_price['STOCK_DATE'] = pd.to_datetime(ds_price['STOCK_DATE'], format='%m/%d/%Y')
ds_price['STOCK_YEAR'] = ds_price['STOCK_DATE'].dt.year
ds_price['STOCK_QUARTER'] = ds_price['STOCK_YEAR'].astype('str') + ds_price['STOCK_DATE'].dt.quarter.astype('str')
ds_price['STOCK_CLOSE'] = ds_price['STOCK_CLOSE'].str.replace(',', '').astype(float)

ds_price = ds_price.sort_values(['STOCK_DATE'], ascending=[False]).groupby(['STOCK_CODE', 'STOCK_QUARTER']).nth(0)
ds_price['NEXT-PRICE-Q'] = ds_price.groupby(level=0)['STOCK_CLOSE'].shift(-1)
ds_price = ds_price.sort_values(['STOCK_DATE'], ascending=[False]).groupby(['STOCK_CODE', 'STOCK_YEAR']).nth(0)
ds_price['NEXT-PRICE-Y'] = ds_price.groupby(level=0)['STOCK_CLOSE'].shift(-1)

ds_ratio = ds_ratio.merge(ds_price, left_on=['STOCK_CODE', 'STOCK_YEAR'], right_on=['STOCK_CODE', 'STOCK_YEAR'], how='right')
ds_ratio['XP'] = ds_ratio['STOCK_DATA'].div(ds_ratio['NEXT-PRICE-Q'])
ds_ratio['RETURN'] = ds_ratio['NEXT-PRICE-Y'].div(ds_ratio['NEXT-PRICE-Q'])
ds_ratio['RANK'] = ds_ratio.groupby(['STOCK_FACTOR', 'STOCK_YEAR'])['XP'].rank('dense', ascending=True)

ds_p = ds_ratio.pivot_table(index=['STOCK_YEAR', 'STOCK_CODE'], columns='STOCK_FACTOR', values=['XP', 'RANK', 'RETURN'])
ds_p.columns = [' '.join(col).strip() for col in ds_p.columns.values]
ds_p = ds_p.reset_index()

ds_p['TMP'] = 1
ds_weight['TMP'] = 1
ds_p = pd.merge(ds_p, ds_weight, on='TMP', how='outer')
#ds_p.fillna(0, inplace=True)

ds_p['RANK'] = ds_p['RANK BVS'] * ds_p['W1'] #+ ds_p['RANK EPS (VND)'] * ds_p['W2']


ds_p['DECILE'] = (ds_p.groupby(['W1', 'STOCK_YEAR'])['RANK']
                                .apply(lambda x: pd.qcut(x,10,duplicates='drop',labels=False)))

ds_p = ds_p.loc[ds_p['DECILE'] == 9.0]

ds_p['RETURN'] = ds_p['RETURN BVS']
ds_p = ds_p.drop(columns=['STOCK_CODE', 'RANK BVS', 'RETURN BVS'])
ds_p = ds_p.drop(columns=['TMP', 'RANK', 'DECILE'])

s6 = ds_p.groupby(['W1', 'STOCK_YEAR']).agg({'RETURN':'mean'})
s6 = s6.reset_index()

s6.pivot_table(index=(['W1']),columns='STOCK_YEAR', values='RETURN')#.sort_values(by=2017, ascending=False)

#ds_p.to_excel('result_.xlsx', header=None, index=None)
#ds_p

1--- 2.274169445037842 seconds ---


STOCK_YEAR,2011,2012,2013,2014,2015,2016,2017
W1,,,,,,,
1.0,1.253681,1.752102,1.518855,1.585736,1.196865,1.282467,0.885472


In [19]:
import sqlite3
import pandas as pd
import time

st = time.time()

conn = sqlite3.connect("stock.db")
ds_ratio = pd.read_sql_query('select STOCK_CODE, STOCK_FACTOR, STOCK_YEAR, STOCK_DATA from VCSC_FINANCIAL_RATIO', conn)
ds_price = pd.read_sql_query('select STOCK_CODE, STOCK_DATE, STOCK_CLOSE from VCSC_STOCK_PRICE', conn)
conn.close()

print("1--- %s seconds ---" % (time.time() - st))
st = time.time()

#ds_ratio = ds_ratio.loc[ds_ratio['STOCK_FACTOR'] == 'EPS (VND)']
ds_ratio = ds_ratio.loc[ds_ratio['STOCK_FACTOR'] == 'BVS']

ds_ratio = ds_ratio.loc[ds_ratio['STOCK_YEAR'].str.contains('TTM')==False]
ds_ratio['STOCK_DATA'] = ds_ratio['STOCK_DATA'].str.replace('-', '')
ds_ratio['STOCK_DATA'] = ds_ratio['STOCK_DATA'].str.replace(',', '')
ds_ratio['STOCK_DATA'] = pd.to_numeric(ds_ratio['STOCK_DATA'])
ds_ratio['STOCK_YEAR'] = pd.to_numeric(ds_ratio['STOCK_YEAR'])

print("2--- %s seconds ---" % (time.time() - st))
st = time.time()

ds_price['STOCK_DATE'] = pd.to_datetime(ds_price['STOCK_DATE'], format='%m/%d/%Y')
ds_price['STOCK_YEAR'] = ds_price['STOCK_DATE'].dt.year
ds_price['STOCK_QUARTER'] = ds_price['STOCK_YEAR'].astype('str') + ds_price['STOCK_DATE'].dt.quarter.astype('str')
ds_price['STOCK_CLOSE'] = ds_price['STOCK_CLOSE'].str.replace(',', '').astype(float)

print("3--- %s seconds ---" % (time.time() - st))
st = time.time()

ds_price = ds_price.sort_values(['STOCK_DATE'], ascending=[False]).groupby(['STOCK_CODE', 'STOCK_QUARTER']).nth(0)
ds_price['NEXT-PRICE-Q'] = ds_price.groupby(level=0)['STOCK_CLOSE'].shift(-1)
ds_price = ds_price.sort_values(['STOCK_DATE'], ascending=[False]).groupby(['STOCK_CODE', 'STOCK_YEAR']).nth(0)
ds_price['NEXT-PRICE-Y'] = ds_price.groupby(level=0)['STOCK_CLOSE'].shift(-1)

print("4--- %s seconds ---" % (time.time() - st))
st = time.time()

ds_ratio = ds_ratio.merge(ds_price, left_on=['STOCK_CODE', 'STOCK_YEAR'], right_on=['STOCK_CODE', 'STOCK_YEAR'], how='right')
ds_ratio['EP'] = ds_ratio['STOCK_DATA'].div(ds_ratio['NEXT-PRICE-Q'])
ds_ratio['RETURN'] = ds_ratio['NEXT-PRICE-Y'].div(ds_ratio['NEXT-PRICE-Q'])
ds_ratio['DECILE'] = (ds_ratio.groupby('STOCK_YEAR')['EP']
                                .apply(lambda x: pd.qcut(x,10,duplicates='drop',labels=False)))
ds_ratio = ds_ratio.loc[ds_ratio['STOCK_FACTOR'].str.contains('BVS')==True]
'''
s1 = ds_ratio.groupby(['STOCK_YEAR','DECILE'], as_index=False).mean()

print("5--- %s seconds ---" % (time.time() - st))
st = time.time()

s1.pivot(index='DECILE', columns='STOCK_YEAR', values='RETURN')
'''
ds_ratio.to_excel('result.xlsx', header=None, index=None)
ds_ratio

1--- 2.1758604049682617 seconds ---
2--- 0.1748948097229004 seconds ---
3--- 4.908275365829468 seconds ---
4--- 0.9270458221435547 seconds ---


,STOCK_CODE,STOCK_FACTOR,STOCK_YEAR,STOCK_DATA,NEXT-PRICE-Q,STOCK_CLOSE,STOCK_DATE,NEXT-PRICE-Y,EP,RETURN,DECILE
0,XHC,BVS,2016,10576.0,13184.0,11230.0,2016-12-30,15913.0,0.802184,1.206993,2.0
1,XHC,BVS,2017,12997.0,NaN,15913.0,2017-06-30,NaN,NaN,NaN,NaN
2,YBC,BVS,2014,3665.0,3200.0,2500.0,2014-12-31,2200.0,1.145312,0.687500,3.0
3,YBC,BVS,2015,5720.0,3600.0,2200.0,2015-12-31,3400.0,1.588889,0.944444,6.0
4,YBC,BVS,2016,6089.0,NaN,3400.0,2016-07-01,NaN,NaN,NaN,NaN
5,XPH,BVS,2014,16866.0,15300.0,10700.0,2014-12-31,8451.0,1.102353,0.552353,2.0
6,XPH,BVS,2015,16154.0,11057.0,8451.0,2015-12-31,6200.0,1.460975,0.560731,5.0
7,XMD,BVS,2015,12401.0,9659.0,9659.0,2015-12-31,9659.0,1.283880,1.000000,4.0
8,XMD,BVS,2016,13174.0,NaN,9659.0,2016-07-01,NaN,NaN,NaN,NaN
9,VIT,BVS,2017,12498.0,15300.0,18000.0,2017-12-29,13400.0,0.816863,0.875817,3.0


In [1]:
import sqlite3
import pandas as pd
import time

st = time.time()

conn = sqlite3.connect("stock.db")
ds_ratio = pd.read_sql_query('select STOCK_CODE, STOCK_FACTOR, STOCK_YEAR, STOCK_DATA from VCSC_FINANCIAL_RATIO', conn)
ds_price = pd.read_sql_query('select STOCK_CODE, STOCK_DATE, STOCK_CLOSE from VCSC_STOCK_PRICE', conn)
conn.close()

print("1--- %s seconds ---" % (time.time() - st))
st = time.time()

#ds_ratio = ds_ratio.loc[ds_ratio['STOCK_FACTOR'] == 'EPS (VND)']
ds_ratio = ds_ratio.loc[ds_ratio['STOCK_FACTOR'] == 'BVS']

ds_ratio = ds_ratio.loc[ds_ratio['STOCK_YEAR'].str.contains('TTM')==False]
ds_ratio['STOCK_DATA'] = ds_ratio['STOCK_DATA'].str.replace('-', '')
ds_ratio['STOCK_DATA'] = ds_ratio['STOCK_DATA'].str.replace(',', '')
ds_ratio['STOCK_DATA'] = pd.to_numeric(ds_ratio['STOCK_DATA'])
ds_ratio['STOCK_YEAR'] = pd.to_numeric(ds_ratio['STOCK_YEAR'])

print("2--- %s seconds ---" % (time.time() - st))
st = time.time()

ds_price['STOCK_DATE'] = pd.to_datetime(ds_price['STOCK_DATE'], format='%m/%d/%Y')
ds_price['STOCK_YEAR'] = ds_price['STOCK_DATE'].dt.year
ds_price['STOCK_QUARTER'] = ds_price['STOCK_YEAR'].astype('str') + ds_price['STOCK_DATE'].dt.quarter.astype('str')
ds_price['STOCK_CLOSE'] = ds_price['STOCK_CLOSE'].str.replace(',', '').astype(float)

print("3--- %s seconds ---" % (time.time() - st))
st = time.time()

ds_price = ds_price.sort_values(['STOCK_DATE'], ascending=[False]).groupby(['STOCK_CODE', 'STOCK_QUARTER']).nth(0)
ds_price['NEXT-PRICE-Q'] = ds_price.groupby(level=0)['STOCK_CLOSE'].shift(-1)
ds_price = ds_price.sort_values(['STOCK_DATE'], ascending=[False]).groupby(['STOCK_CODE', 'STOCK_YEAR']).nth(0)
ds_price['NEXT-PRICE-Y'] = ds_price.groupby(level=0)['STOCK_CLOSE'].shift(-1)

print("4--- %s seconds ---" % (time.time() - st))
st = time.time()

ds_ratio = ds_ratio.merge(ds_price, left_on=['STOCK_CODE', 'STOCK_YEAR'], right_on=['STOCK_CODE', 'STOCK_YEAR'], how='right')
ds_ratio['EP'] = ds_ratio['STOCK_DATA'].div(ds_ratio['NEXT-PRICE-Q'])
ds_ratio['RETURN'] = ds_ratio['NEXT-PRICE-Y'].div(ds_ratio['NEXT-PRICE-Q'])
ds_ratio['DECILE'] = (ds_ratio.groupby('STOCK_YEAR')['EP']
                                .apply(lambda x: pd.qcut(x,10,duplicates='drop',labels=False)))
#s1 = ds_ratio.groupby(['STOCK_YEAR','DECILE'], as_index=False).mean()

print("5--- %s seconds ---" % (time.time() - st))
st = time.time()
#s1
#s1.pivot(index='DECILE', columns='STOCK_YEAR', values='RETURN')
ds_ratio

1--- 2.4082252979278564 seconds ---
2--- 0.14513087272644043 seconds ---
3--- 4.3364293575286865 seconds ---
4--- 0.8317775726318359 seconds ---
5--- 0.08577895164489746 seconds ---


,STOCK_CODE,STOCK_FACTOR,STOCK_YEAR,STOCK_DATA,NEXT-PRICE-Q,STOCK_CLOSE,STOCK_DATE,NEXT-PRICE-Y,EP,RETURN,DECILE
0,XHC,BVS,2016,10576.0,13184.0,11230.0,2016-12-30,15913.0,0.802184,1.206993,2.0
1,XHC,BVS,2017,12997.0,NaN,15913.0,2017-06-30,NaN,NaN,NaN,NaN
2,YBC,BVS,2014,3665.0,3200.0,2500.0,2014-12-31,2200.0,1.145312,0.687500,3.0
3,YBC,BVS,2015,5720.0,3600.0,2200.0,2015-12-31,3400.0,1.588889,0.944444,6.0
4,YBC,BVS,2016,6089.0,NaN,3400.0,2016-07-01,NaN,NaN,NaN,NaN
5,XPH,BVS,2014,16866.0,15300.0,10700.0,2014-12-31,8451.0,1.102353,0.552353,2.0
6,XPH,BVS,2015,16154.0,11057.0,8451.0,2015-12-31,6200.0,1.460975,0.560731,5.0
7,XMD,BVS,2015,12401.0,9659.0,9659.0,2015-12-31,9659.0,1.283880,1.000000,4.0
8,XMD,BVS,2016,13174.0,NaN,9659.0,2016-07-01,NaN,NaN,NaN,NaN
9,VIT,BVS,2017,12498.0,15300.0,18000.0,2017-12-29,13400.0,0.816863,0.875817,3.0


In [2]:
import sqlite3
import pandas as pd
import time

st = time.time()

conn = sqlite3.connect("stock.db")
ds_ratio = pd.read_sql_query('select STOCK_CODE, STOCK_FACTOR, STOCK_YEAR, STOCK_DATA from VCSC_FINANCIAL_RATIO', conn)
ds_price = pd.read_sql_query('select STOCK_CODE, STOCK_DATE, STOCK_CLOSE from VCSC_STOCK_PRICE', conn)
conn.close()

print("1--- %s seconds ---" % (time.time() - st))
st = time.time()

ds_ratio = ds_ratio.loc[ds_ratio['STOCK_FACTOR'] == 'EPS (VND)']
#ds_ratio = ds_ratio.loc[ds_ratio['STOCK_FACTOR'] == 'BVS']

ds_ratio = ds_ratio.loc[ds_ratio['STOCK_YEAR'].str.contains('TTM')==False]
ds_ratio['STOCK_DATA'] = ds_ratio['STOCK_DATA'].str.replace('-', '')
ds_ratio['STOCK_DATA'] = ds_ratio['STOCK_DATA'].str.replace(',', '')
ds_ratio['STOCK_DATA'] = pd.to_numeric(ds_ratio['STOCK_DATA'])
ds_ratio['STOCK_YEAR'] = pd.to_numeric(ds_ratio['STOCK_YEAR'])

print("2--- %s seconds ---" % (time.time() - st))
st = time.time()

ds_price['STOCK_DATE'] = pd.to_datetime(ds_price['STOCK_DATE'], format='%m/%d/%Y')
ds_price['STOCK_YEAR'] = ds_price['STOCK_DATE'].dt.year
ds_price['STOCK_QUARTER'] = ds_price['STOCK_YEAR'].astype('str') + ds_price['STOCK_DATE'].dt.quarter.astype('str')
ds_price['STOCK_CLOSE'] = ds_price['STOCK_CLOSE'].str.replace(',', '').astype(float)

print("3--- %s seconds ---" % (time.time() - st))
st = time.time()

ds_price = ds_price.sort_values(['STOCK_DATE'], ascending=[False]).groupby(['STOCK_CODE', 'STOCK_QUARTER']).nth(0)
ds_price['NEXT-PRICE-Q'] = ds_price.groupby(level=0)['STOCK_CLOSE'].shift(-1)
ds_price = ds_price.sort_values(['STOCK_DATE'], ascending=[False]).groupby(['STOCK_CODE', 'STOCK_YEAR']).nth(0)
ds_price['NEXT-PRICE-Y'] = ds_price.groupby(level=0)['STOCK_CLOSE'].shift(-1)

print("4--- %s seconds ---" % (time.time() - st))
st = time.time()

ds_ratio = ds_ratio.merge(ds_price, left_on=['STOCK_CODE', 'STOCK_YEAR'], right_on=['STOCK_CODE', 'STOCK_YEAR'], how='right')
ds_ratio['EP'] = ds_ratio['STOCK_DATA'].div(ds_ratio['NEXT-PRICE-Q'])
ds_ratio['RETURN'] = ds_ratio['NEXT-PRICE-Y'].div(ds_ratio['NEXT-PRICE-Q'])
ds_ratio['DECILE'] = (ds_ratio.groupby('STOCK_YEAR')['EP']
                                .apply(lambda x: pd.qcut(x,10,duplicates='drop',labels=False)))
s1 = ds_ratio.groupby(['STOCK_YEAR','DECILE'], as_index=False).mean()

print("5--- %s seconds ---" % (time.time() - st))
st = time.time()

s1.pivot(index='DECILE', columns='STOCK_YEAR', values='RETURN')

1--- 2.9041662216186523 seconds ---
2--- 0.19501090049743652 seconds ---
3--- 7.320418834686279 seconds ---
4--- 1.3220758438110352 seconds ---
5--- 0.03200173377990723 seconds ---


STOCK_YEAR,2011,2012,2013,2014,2015,2016,2017
DECILE,,,,,,,
0.0,0.837704,1.115118,0.995606,0.925045,0.885809,1.107489,0.926600
1.0,0.920135,1.084825,1.175648,1.014271,1.094995,1.135392,0.887672
2.0,0.857687,1.184520,0.969740,1.067192,1.039543,1.164843,0.903127
3.0,0.952850,1.267875,0.991406,1.126042,1.083746,1.091764,0.938852
4.0,1.043203,1.271877,1.062791,1.098411,1.080078,1.151107,0.940865
5.0,1.119233,1.301727,1.098700,1.146277,1.141121,1.275363,0.969936
6.0,1.043732,1.284205,1.106619,1.245019,1.146367,1.149280,0.946592
7.0,1.119494,1.407103,1.096132,1.144480,1.094093,1.042661,0.959581
8.0,1.181776,1.397383,1.231342,1.302798,1.205132,1.103859,0.959370
